In [ ]:
# ADD columns
ALTER TABLE ml.web_data ADD COLUMN extracted_by VARCHAR ;

## ADD columns defaulr boolean
ALTER TABLE ml.web_data ADD COLUMN mobile_matching BOOLEAN DEFAULT FALSE ;
ALTER TABLE ml.web_data ADD COLUMN mobile_matching VARCHAR(20);


## Rename column
ALTER TABLE ml.web_data RENAME COLUMN mobile_match  to mobile_matching ;

## change types ( int to str)
ALTER TABLE ml.inquiry_data ALTER COLUMN mobile TYPE VARCHAR(10) ;

## Update
UPDATE ml.web_data SET mobile_matching = True where mobile_match = 'Yes' ;

## Drop columns
ALTER TABLE ml.web_data DROP COLUMN mobile_match ;

## Change column length
ALTER TABLE ml.web_data ALTER COLUMN contact_person TYPE VARCHAR(40) ;

## The basic syntax of ALTER TABLE to ADD UNIQUE CONSTRAINT to a table is as
ALTER  TABLE table_name ADD CONSTRAINT myuniquecontraint UNIQUE(column1, column2,..) ;
ALTER TABLE ml.web_data ADD CONSTRAINT company_contact_number_pincode UNIQUE (company_name,contact_number,pincode);

## create column index postgresql:
# A single-column index is one that is created based on only one table column
CREATE INDEX  ON ml.ec_rfm_tim(profile_id) ;

# Multiple column index
CREATE INDEX index_name ON table_name(column1_name, column2_name) ;

## DElete
Delete from ml.web_data where extracted_by = 'Ayush' and generated = 1656077468 and  pincode = 0 ;

## search all the table in ml schema
\dt *ml.*                           # archival

\dt *.*facebook*                    # 

In [ ]:
ALTER TABLE ml.ec_rfm_tim RENAME COLUMN "vintage(year)"  to vintage_year ;

In [ ]:
# update query
from datetime import date
import os
todays_date = date.today()
day = todays_date.day
month = todays_date.month

file_name = 'keyword_update_'+str(day)+'_'+str(month)+'.csv'
path_to_create = os.path.join(f'/mnt/data/rishabh/sqlanalysis/sql_new', file_name)

for i in tqdm(range(d5.shape[0])):
    if d5.iloc[i,2]==False:
        data = d5.iloc[i]
        query = f"""update ml.web_data
        set keyword='{data['keyword']}', updated_at = {round(datetime.datetime.now().timestamp())}
        where meteric_id = {data['deleted_metric_id']} """
        try:
            with engine.connect() as con:
                con.execute(sqlalchemy.text(query))
            d5.iloc[i,2]=True
            if i%100 == 0:
                d5.to_csv(path_to_create, index = False)
        except Exception as e:
            print('x',e,end='')
d5.to_csv(path_to_create, index = False)

In [ ]:
# update query
query = f"""update ml.web_data
        set selected = True
        where meteric_id in  """ + str(tuple(rest_of_data_insert.meteric_id)) + """ ; """
# print(query)
with engine.connect() as con:
    try:
        con.execute(sqlalchemy.text(query))
    except Exception as e:
            print('x',e,end='')

In [ ]:
# insert query
with engine.connect() as con:
    try:
        rest_of_data_insert.to_sql('web_data', con, schema='ml', if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)    
    except sqlalchemy.exc.StatementError as e:
        print('Error occured while executing a query {}'.format(e.args))
    except:
        print ('Some error occured while insertion ')

In [ ]:
# Delete query
query = f""" Delete from ml.web_data
        where meteric_id in  {tuple(d.meteric_id)} """
print(query)
with engine.connect() as con:
    try:
        con.execute(sqlalchemy.text(query))
    except Exception as e:
            print('x',e,end='')